In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
import math

In [ ]:
t_data = pd.DataFrame()

combined_news_djia = pd.read_csv('../data/Combined_News_DJIA.csv')
combined_news_djia['Top1'] = combined_news_djia['Top1'].apply(lambda x: x[2:-1] if x[0]=='b' else x)
combined_news_djia['Top2'] = combined_news_djia['Top2'].apply(lambda x: x[2:-1] if x[0]=='b' else x)

t_data['Text'] = combined_news_djia['Top1'] + " " + combined_news_djia['Top2']
t_data['Date'] = combined_news_djia['Date']
t_data.set_index('Date')

nltk.download('stopwords')
tokenizer = WordPunctTokenizer()
stop_words = set(stopwords.words('english'))

def process_headline(x):
    return " ".join([w.lower() for w in tokenizer.tokenize(x) if not w.lower() in stop_words])

t_data['Text'] = t_data['Text'].apply(process_headline)
t_data

In [ ]:
import fasttext.util

# Download FastText word vectors
# fasttext.util.download_model('en', if_exists='ignore')  # Download English language embeddings
ft = fasttext.load_model('../../NLPstockPredictions/cc.en.300.bin')  # Load the downloaded model

def get_embeddings(data):
    combo = []
    for row in data.values:
        news_embedding = np.mean([ft.get_word_vector(word) for word in row[0].split()], axis=0)
        combo.append(news_embedding)
    return np.array(combo), data.values[:, 1]


In [ ]:
t_data = get_embeddings(t_data)
with open('../fasttext_embeddings.npy', 'wb') as f:
    np.save(f, t_data[0])